In [22]:
# Setup LangSmith for tracing and debugging
import os
from pathlib import Path
from deepagents.backends.filesystem import FilesystemBackend
from langchain_openai import ChatOpenAI
import uuid
from deepagents import create_deep_agent
from langgraph.checkpoint.memory import MemorySaver

# Load environment variables
root_dir = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
env_path = root_dir / ".env"
env = {}

if env_path.exists():
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        env[key.strip()] = value.strip()

# Configure LangSmith
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = env.get("LANGSMITH_API_KEY", os.environ.get("LANGSMITH_API_KEY", ""))
os.environ["LANGSMITH_PROJECT"] = "finer-skills-testing"

# Verify LangSmith is configured
if os.environ.get("LANGSMITH_API_KEY"):
    print("✓ LangSmith enabled for tracing")
    print(f"  Project: {os.environ['LANGSMITH_PROJECT']}")
else:
    print("⚠ LangSmith API key not found - tracing disabled")

✓ LangSmith enabled for tracing
  Project: finer-skills-testing


In [25]:
checkpointer = MemorySaver()

model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_deep_agent(
    model,
    # system_prompt=(
    #     "You are a SQL query assistant that helps users "
    #     "write queries against business databases."
    # ),
    skills=["/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/"],
    backend=FilesystemBackend(root_dir="/home/snt/projects_lujun/LabAgentSkill/skillsHub"),
    checkpointer=checkpointer,
    interrupt_on={
        "write_file": True,  # Default: approve, edit, reject
        "read_file": False,  # No interrupts needed
        "edit_file": True    # Default: approve, edit, reject
    },
)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}


In [26]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Find the most suitable Skill first and use them to make decision for the following instruction. "
                    "Write a SQL query to find all customers who made orders over $1000 in the last month."
                ),
            }
        ]
    },
    config
)

In [27]:
# LangSmith Test Results Analysis
print("\n" + "="*70)
print("LANGSMITH TRACE ANALYSIS")
print("="*70)

# Extract run information
if hasattr(result, '__dict__'):
    print("\n📊 Result Structure:")
    print(f"  Keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dict'}")

# Display LangSmith trace URL if available
if os.environ.get("LANGSMITH_API_KEY"):
    print(f"\n🔍 View detailed trace in LangSmith:")
    print(f"  Project: {os.environ['LANGSMITH_PROJECT']}")
    print(f"  Dashboard: https://smith.langchain.com/")
    print(f"\n  Each run is automatically logged with:")
    print(f"    • Agent input/output")
    print(f"    • Tool calls and results")
    print(f"    • Token usage")
    print(f"    • Execution time")
    print(f"    • Error tracking")

# Summary stats
print(f"\n📈 Test Summary:")
print(f"  Total messages: {len(result.get('messages', []))}")


LANGSMITH TRACE ANALYSIS

🔍 View detailed trace in LangSmith:
  Project: finer-skills-testing
  Dashboard: https://smith.langchain.com/

  Each run is automatically logged with:
    • Agent input/output
    • Tool calls and results
    • Token usage
    • Execution time
    • Error tracking

📈 Test Summary:
  Total messages: 4


In [1]:
# Add project to Python path to import LabAgentSkill module
import sys
from pathlib import Path

project_root = Path.cwd()
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"✓ Added to sys.path: {src_path}")

from LabAgentSkill import skills_utils

# Test the function
skills_data = skills_utils.read_all_skills_metadata("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills")
skills_utils.display_skills_metadata(skills_data)

✓ Added to sys.path: /home/snt/projects_lujun/LabAgentSkill/src

████████████████████████████████████████████████████████████████████████████████████████████████████
█                                                                                                  █
█                             SKILLS METADATA SUMMARY (2 skills found)                             █
█                                                                                                  █
████████████████████████████████████████████████████████████████████████████████████████████████████

┌─ [1] langgraph-docs
│
│  📝 Description
│     Use this skill for requests related to LangGraph in order to fetch relevant documenta
│     ti...
│  📄 Body:
│     # langgraph-docs  ## Overview  This skill explains how to access LangGraph Py...
│  📁 Path:               /home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/langgraph-docs
└─────────────────────────────────────────────────────────────────────────────────────────